In [86]:
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC  # Importando SVM
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn import metrics
from random import shuffle
import numpy as np
import pandas as pd

In [87]:
column_names = [
    'class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
    'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
    'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 
    'stalk-surface-below-ring', 'stalk-color-above-ring', 
    'stalk-color-below-ring', 'veil-type', 'veil-color',
    'ring-number', 'ring-type', 'spore-print-color', 
    'population', 'habitat'
]

df = pd.read_csv('agaricus-lepiota.data', names=column_names)

print(f"Shape do dataset: {df.shape}")
print(f"\nPrimeiras 5 linhas:")
print(df.head())

Shape do dataset: (8124, 23)

Primeiras 5 linhas:
  class cap-shape cap-surface cap-color bruises odor gill-attachment  \
0     p         x           s         n       t    p               f   
1     e         x           s         y       t    a               f   
2     e         b           s         w       t    l               f   
3     p         x           y         w       t    p               f   
4     e         x           s         g       f    n               f   

  gill-spacing gill-size gill-color  ... stalk-surface-below-ring  \
0            c         n          k  ...                        s   
1            c         b          k  ...                        s   
2            c         b          n  ...                        s   
3            c         n          n  ...                        s   
4            w         b          k  ...                        s   

  stalk-color-above-ring stalk-color-below-ring veil-type veil-color  \
0                      w      

In [88]:
print("=== VERIFICAÇÃO DE DADOS FALTANTES ===")
print(f"Total de valores ausentes por coluna:")
missing_values = df.isnull().sum()
print(missing_values)

print(f"\nVerificando valores '?' que representam dados faltantes:")
question_marks = (df == '?').sum()
print(question_marks)

print(f"\nTotal de linhas com dados faltantes:")
rows_with_missing = df[df == '?'].any(axis=1).sum()
print(f"{rows_with_missing} linhas contém '?'")

print(f"\nValores únicos em 'stalk-root':")
print(df['stalk-root'].value_counts())

=== VERIFICAÇÃO DE DADOS FALTANTES ===
Total de valores ausentes por coluna:
class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

Verificando valores '?' que representam dados faltantes:
class                          0
cap-shape                      0
cap-surface                    0
cap-color                      0
bruises                       

In [89]:
print("=== REMOÇÃO DE DADOS FALTANTES ===")
print(f"Dataset original: {df.shape}")

df_clean = df[~(df == '?').any(axis=1)]

print(f"Dataset após remoção: {df_clean.shape}")
print(f"Linhas removidas: {df.shape[0] - df_clean.shape[0]}")

print(f"\nVerificação final - valores '?' restantes:")
print((df_clean == '?').sum())

=== REMOÇÃO DE DADOS FALTANTES ===
Dataset original: (8124, 23)
Dataset após remoção: (5644, 23)
Linhas removidas: 2480

Verificação final - valores '?' restantes:
class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64
Dataset após remoção: (5644, 23)
Linhas removidas: 2480

Verificação final - valores '?' restantes:
class                       0
cap

In [90]:
print("=== PREPARAÇÃO DOS DADOS ===")
X = df_clean.drop('class', axis=1)
y = df_clean['class']

print(f"Classes no target:")
print(y.value_counts())

label_encoders = {}
x_encoded = X.copy()

for column in X.columns:
    le = LabelEncoder()
    x_encoded[column] = le.fit_transform(X[column])
    label_encoders[column] = le
    
le_target = LabelEncoder()
y_encoded = le_target.fit_transform(y)

print(f"\nShape dos dados finais:")
print(f"X: {x_encoded.shape}")
print(f"y: {y_encoded.shape}")
print(f"Mapeamento do target: {dict(zip(le_target.classes_, le_target.transform(le_target.classes_)))}")

=== PREPARAÇÃO DOS DADOS ===
Classes no target:
class
e    3488
p    2156
Name: count, dtype: int64



Shape dos dados finais:
X: (5644, 22)
y: (5644,)
Mapeamento do target: {'e': np.int64(0), 'p': np.int64(1)}


In [91]:
print("=== SHUFFLE DOS DADOS ===")

x_array = x_encoded.values
y_array = y_encoded

indices = np.arange(len(x_array))
np.random.shuffle(indices)

x_shuffled = x_array[indices]
y_shuffled = y_array[indices]

print(f"Dados embaralhados com sucesso!")
print(f"Primeiros 5 indices após shuffle: {indices[:5]}")

=== SHUFFLE DOS DADOS ===
Dados embaralhados com sucesso!
Primeiros 5 indices após shuffle: [3215 1743 1709 3211 1098]


In [92]:
print("=== DIVISÃO TREINO/TESTE ===")

x_train, x_test, y_train, y_test = train_test_split(
    x_shuffled, y_shuffled,
    test_size=0.2,
    random_state=42,
    stratify=y_shuffled
)

print(f"Tamanho do conjunto de treino: {x_train.shape}")
print(f"Tamanho do conjunto de teste: {x_test.shape}")
print(f"Distribuição de classes no treino: {np.bincount(y_train)}")
print(f"Distribuição das classes no teste: {np.bincount(y_test)}")


=== DIVISÃO TREINO/TESTE ===
Tamanho do conjunto de treino: (4515, 22)
Tamanho do conjunto de teste: (1129, 22)
Distribuição de classes no treino: [2790 1725]
Distribuição das classes no teste: [698 431]


In [93]:
print("=== TREINAMENTO DO PERCEPTRON ===")

perceptron = Perceptron(random_state=42, max_iter=1000)
perceptron.fit(x_train, y_train)

print(f"Modelo treinado com sucesso!")
print(f"Número de iterações realizadas: {perceptron.n_iter_}")


=== TREINAMENTO DO PERCEPTRON ===
Modelo treinado com sucesso!
Número de iterações realizadas: 44
Modelo treinado com sucesso!
Número de iterações realizadas: 44


In [94]:
print("=== AVALIAÇÃO DO MODELO ===")
y_pred_train = perceptron.predict(x_train)
y_pred_test = perceptron.predict(x_test)

print(f"Acurácia no treino: {accuracy_score(y_train, y_pred_train):.4f}")
print(f"Acurácia no teste: {accuracy_score(y_test, y_pred_test):.4f}")

print(f"\nRelatório de classificação (teste):")
target_names = ['Comestível', 'Venenoso']
print(classification_report(y_test, y_pred_test, target_names=target_names))

print(f"\nMatriz de confusão (teste):")
cm = confusion_matrix(y_test, y_pred_test)
print(cm)
print(f"Verdadeiros Positivos (Venenosos corretamente identificados): {cm[1,1]}")
print(f"Falsos Negativos (Venenosos classificados como comestíveis): {cm[1,0]}")


=== AVALIAÇÃO DO MODELO ===
Acurácia no treino: 0.8560
Acurácia no teste: 0.8441

Relatório de classificação (teste):
              precision    recall  f1-score   support

  Comestível       0.99      0.75      0.86       698
    Venenoso       0.71      0.99      0.83       431

    accuracy                           0.84      1129
   macro avg       0.85      0.87      0.84      1129
weighted avg       0.89      0.84      0.85      1129


Matriz de confusão (teste):
[[525 173]
 [  3 428]]
Verdadeiros Positivos (Venenosos corretamente identificados): 428
Falsos Negativos (Venenosos classificados como comestíveis): 3


In [95]:
print("=== TREINAMENTO DO SVM ===")

# Criar e treinar o modelo SVM
svm = SVC(random_state=42, kernel='rbf')  # kernel RBF é uma boa opção padrão
svm.fit(x_train, y_train)

print(f"Modelo SVM treinado com sucesso!")
print(f"Kernel usado: {svm.kernel}")
print(f"Número de vetores de suporte: {svm.n_support_}")
print(f"Total de vetores de suporte: {svm.support_vectors_.shape[0]}")

=== TREINAMENTO DO SVM ===


Modelo SVM treinado com sucesso!
Kernel usado: rbf
Número de vetores de suporte: [241 240]
Total de vetores de suporte: 481


In [96]:
print("=== AVALIAÇÃO DO SVM ===")

# Predições do SVM
y_pred_train_svm = svm.predict(x_train)
y_pred_test_svm = svm.predict(x_test)

print(f"Acurácia SVM no treino: {accuracy_score(y_train, y_pred_train_svm):.4f}")
print(f"Acurácia SVM no teste: {accuracy_score(y_test, y_pred_test_svm):.4f}")

print(f"\nRelatório de classificação SVM (teste):")
target_names = ['Comestível', 'Venenoso']
print(classification_report(y_test, y_pred_test_svm, target_names=target_names))

print(f"\nMatriz de confusão SVM (teste):")
cm_svm = confusion_matrix(y_test, y_pred_test_svm)
print(cm_svm)
print(f"Verdadeiros Positivos (Venenosos corretamente identificados): {cm_svm[1,1]}")
print(f"Falsos Negativos (Venenosos classificados como comestíveis): {cm_svm[1,0]}")

=== AVALIAÇÃO DO SVM ===
Acurácia SVM no treino: 0.9976
Acurácia SVM no teste: 1.0000

Relatório de classificação SVM (teste):
              precision    recall  f1-score   support

  Comestível       1.00      1.00      1.00       698
    Venenoso       1.00      1.00      1.00       431

    accuracy                           1.00      1129
   macro avg       1.00      1.00      1.00      1129
weighted avg       1.00      1.00      1.00      1129


Matriz de confusão SVM (teste):
[[698   0]
 [  0 431]]
Verdadeiros Positivos (Venenosos corretamente identificados): 431
Falsos Negativos (Venenosos classificados como comestíveis): 0
Acurácia SVM no treino: 0.9976
Acurácia SVM no teste: 1.0000

Relatório de classificação SVM (teste):
              precision    recall  f1-score   support

  Comestível       1.00      1.00      1.00       698
    Venenoso       1.00      1.00      1.00       431

    accuracy                           1.00      1129
   macro avg       1.00      1.00      1.

In [97]:
print("=== COMPARAÇÃO PERCEPTRON vs SVM ===")

# Comparar as acurácias
acc_perceptron_train = accuracy_score(y_train, y_pred_train)
acc_perceptron_test = accuracy_score(y_test, y_pred_test)
acc_svm_train = accuracy_score(y_train, y_pred_train_svm)
acc_svm_test = accuracy_score(y_test, y_pred_test_svm)

print(f"{'Modelo':<12} {'Treino':<8} {'Teste':<8}")
print(f"{'='*12} {'='*8} {'='*8}")
print(f"{'Perceptron':<12} {acc_perceptron_train:<8.4f} {acc_perceptron_test:<8.4f}")
print(f"{'SVM':<12} {acc_svm_train:<8.4f} {acc_svm_test:<8.4f}")

print(f"\nDiferença de performance:")
diff_train = acc_svm_train - acc_perceptron_train
diff_test = acc_svm_test - acc_perceptron_test
print(f"Treino: SVM {'supera' if diff_train > 0 else 'fica atrás do'} Perceptron em {abs(diff_train):.4f}")
print(f"Teste: SVM {'supera' if diff_test > 0 else 'fica atrás do'} Perceptron em {abs(diff_test):.4f}")

=== COMPARAÇÃO PERCEPTRON vs SVM ===
Modelo       Treino   Teste   
============ ======== ========
Perceptron   0.8560   0.8441  
SVM          0.9976   1.0000  

Diferença de performance:
Treino: SVM supera Perceptron em 0.1415
Teste: SVM supera Perceptron em 0.1559


In [98]:
print("=== TESTANDO DIFERENTES KERNELS DO SVM ===")

# Testar diferentes kernels
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
results = {}

for kernel in kernels:
    print(f"\nTestando kernel: {kernel}")
    
    # Criar e treinar SVM com kernel específico
    svm_kernel = SVC(random_state=42, kernel=kernel)
    svm_kernel.fit(x_train, y_train)
    
    # Fazer predições
    y_pred_kernel = svm_kernel.predict(x_test)
    acc_kernel = accuracy_score(y_test, y_pred_kernel)
    
    results[kernel] = acc_kernel
    print(f"Acurácia com kernel {kernel}: {acc_kernel:.4f}")

print(f"\n=== RESUMO DOS KERNELS ===")
best_kernel = max(results, key=results.get)
print(f"{'Kernel':<10} {'Acurácia':<10}")
print(f"{'='*10} {'='*10}")
for kernel, acc in sorted(results.items(), key=lambda x: x[1], reverse=True):
    marker = " ⭐" if kernel == best_kernel else ""
    print(f"{kernel:<10} {acc:<10.4f}{marker}")
    
print(f"\nMelhor kernel: {best_kernel} com acurácia de {results[best_kernel]:.4f}")

=== TESTANDO DIFERENTES KERNELS DO SVM ===

Testando kernel: linear
Acurácia com kernel linear: 0.9814

Testando kernel: poly
Acurácia com kernel poly: 1.0000

Testando kernel: rbf
Acurácia com kernel linear: 0.9814

Testando kernel: poly
Acurácia com kernel poly: 1.0000

Testando kernel: rbf
Acurácia com kernel rbf: 1.0000

Testando kernel: sigmoid
Acurácia com kernel rbf: 1.0000

Testando kernel: sigmoid
Acurácia com kernel sigmoid: 0.7857

=== RESUMO DOS KERNELS ===
Kernel     Acurácia  
========== ==========
poly       1.0000     ⭐
rbf        1.0000    
linear     0.9814    
sigmoid    0.7857    

Melhor kernel: poly com acurácia de 1.0000
Acurácia com kernel sigmoid: 0.7857

=== RESUMO DOS KERNELS ===
Kernel     Acurácia  
========== ==========
poly       1.0000     ⭐
rbf        1.0000    
linear     0.9814    
sigmoid    0.7857    

Melhor kernel: poly com acurácia de 1.0000


In [99]:
print("=== Fazendo F1-Score ===")

f1_perceptron = metrics.f1_score(y_test, y_pred_test)
f1_svm = metrics.f1_score(y_test, y_pred_test_svm)

print(f"F1-Score Perceptron: {f1_perceptron:.4f}")
print(f"F1-Score SVM: {f1_svm:.4f}")

=== Fazendo F1-Score ===
F1-Score Perceptron: 0.8295
F1-Score SVM: 1.0000
